In [1]:
import os
os.chdir("../")
%pwd

'c:\\Users\\HP\\Desktop\\ML\\Customer_Churn_Predictor'

In [11]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    learning_rate: float
    l2_leaf_reg: int
    iterations: int
    depth: int
    border_count: int
    bagging_temperature: int
    target_column: str

In [12]:
from ChurnPredictor.constants import *
from ChurnPredictor.utils.common import read_yaml, create_directories

In [13]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.CatBoostClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            learning_rate = params.learning_rate,
            l2_leaf_reg = params.l2_leaf_reg,
            iterations = params.iterations,
            depth = params.depth,
            border_count = params.border_count,
            bagging_temperature = params.bagging_temperature,
            target_column = schema.name
            
        )

        return model_trainer_config

In [14]:
import pandas as pd
import os
from ChurnPredictor import logger
from catboost import CatBoostClassifier
import joblib

In [15]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]


        cb = CatBoostClassifier(learning_rate=self.config.learning_rate, l2_leaf_reg=self.config.l2_leaf_reg, iterations=self.config.iterations, depth=self.config.depth, border_count=self.config.border_count, bagging_temperature=self.config.bagging_temperature)
        cb.fit(train_x, train_y)

        joblib.dump(cb, os.path.join(self.config.root_dir, self.config.model_name))



In [16]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-05-27 10:21:44,276: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-27 10:21:44,280: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-27 10:21:44,285: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-05-27 10:21:44,288: INFO: common: created directory at: artifacts]


[2024-05-27 10:21:44,289: INFO: common: created directory at: artifacts/model_trainer]
0:	learn: 0.6615766	total: 165ms	remaining: 2m 45s
1:	learn: 0.6340090	total: 171ms	remaining: 1m 25s
2:	learn: 0.6072422	total: 175ms	remaining: 58.2s
3:	learn: 0.5851739	total: 182ms	remaining: 45.3s
4:	learn: 0.5639128	total: 187ms	remaining: 37.2s
5:	learn: 0.5464006	total: 191ms	remaining: 31.6s
6:	learn: 0.5292174	total: 199ms	remaining: 28.2s
7:	learn: 0.5137084	total: 203ms	remaining: 25.2s
8:	learn: 0.5000705	total: 209ms	remaining: 23s
9:	learn: 0.4883601	total: 214ms	remaining: 21.2s
10:	learn: 0.4784973	total: 219ms	remaining: 19.7s
11:	learn: 0.4703498	total: 224ms	remaining: 18.5s
12:	learn: 0.4613531	total: 231ms	remaining: 17.5s
13:	learn: 0.4521504	total: 235ms	remaining: 16.6s
14:	learn: 0.4449778	total: 240ms	remaining: 15.7s
15:	learn: 0.4380455	total: 244ms	remaining: 15s
16:	learn: 0.4316226	total: 248ms	remaining: 14.3s
17:	learn: 0.4243405	total: 252ms	remaining: 13.8s
18:	lea